In [0]:
# Removed Access key in order to post in github
spark.conf.set(
    "fs.azure.account.key.goodreadsreviews60104758.dfs.core.windows.net",
    "<Access-key>"
)

In [0]:
# Load training split from features_v2
train_path = "abfss://lakehouse@goodreadsreviews60104758.dfs.core.windows.net/Gold_layer/features_v2/train"
train_df = spark.read.format("delta").load(train_path)

# Show first 5 review texts to verify
train_df.select("review_text").show(5, truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                                         review_text|
+----------------------------------------------------------------------------------------------------+
|this takes place after ink exchange also. it focuses on niall's and irial's relationship. niall i...|
|love melissa marrs wicked lovely books and this novella did not let me down! very interesting and...|
|                                            review to come after i get some new year's eve dinner ;)|
|again... this was very entertaining to read. gives you adding insight into the side characters li...|
|                                                              irial and niall... what else is there?|
+----------------------------------------------------------------------------------------------------+
only showing top 5 rows


In [0]:
from pyspark.sql.functions import col, lower, trim, length, regexp_replace
# Lowercase can review text column : already done in the previous notebook (lab3)
train_df = train_df.withColumn("review_text", lower(col("review_text")))
train_df.select("review_text").show(3, truncate=80)

+--------------------------------------------------------------------------------+
|                                                                     review_text|
+--------------------------------------------------------------------------------+
|this takes place after ink exchange also. it focuses on niall's and irial's r...|
|love melissa marrs wicked lovely books and this novella did not let me down! ...|
|                        review to come after i get some new year's eve dinner ;)|
+--------------------------------------------------------------------------------+
only showing top 3 rows


In [0]:
from pyspark.sql.functions import regexp_replace, trim, col

# Remove punctuation
train_df = train_df.withColumn(
    "review_text",
    trim(regexp_replace(col("review_text"), r"[^\w\s]+", " "))
)

# Replace multiple spaces with a single space
train_df = train_df.withColumn(
    "review_text",
    regexp_replace(col("review_text"), r"\s+", " ")
)

train_df.select("review_text").show(5, truncate=100)


+----------------------------------------------------------------------------------------------------+
|                                                                                         review_text|
+----------------------------------------------------------------------------------------------------+
|this takes place after ink exchange also it focuses on niall s and irial s relationship niall is ...|
|love melissa marrs wicked lovely books and this novella did not let me down very interesting and ...|
|                                               review to come after i get some new year s eve dinner|
|again this was very entertaining to read gives you adding insight into the side characters lives ...|
|                                                                  irial and niall what else is there|
+----------------------------------------------------------------------------------------------------+
only showing top 5 rows


In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import re

# Function to replace URLs and numbers
def replace_urls_numbers(text):
    if text is None:
        return ""
    # Replace URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "<URL>", text, flags=re.MULTILINE)
    # Replace numbers
    text = re.sub(r"\d+", "<NUM>", text)
    return text

url_num_udf = udf(replace_urls_numbers, StringType())
train_df = train_df.withColumn("review_text", url_num_udf(col("review_text")))
train_df.select("review_text").show(5, truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                                         review_text|
+----------------------------------------------------------------------------------------------------+
|this takes place after ink exchange also it focuses on niall s and irial s relationship niall is ...|
|love melissa marrs wicked lovely books and this novella did not let me down very interesting and ...|
|                                               review to come after i get some new year s eve dinner|
|again this was very entertaining to read gives you adding insight into the side characters lives ...|
|                                                                  irial and niall what else is there|
+----------------------------------------------------------------------------------------------------+
only showing top 5 rows


In [0]:
import subprocess, sys
subprocess.check_call([sys.executable, "-m", "pip", "install", "emoji", "-q"])


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


0

In [0]:
# trim the leading and trailing space - already done in lab3
train_df = train_df.withColumn("review_text", trim(col("review_text")))
train_df.select("review_text").show(5, truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                                         review_text|
+----------------------------------------------------------------------------------------------------+
|this takes place after ink exchange also it focuses on niall s and irial s relationship niall is ...|
|love melissa marrs wicked lovely books and this novella did not let me down very interesting and ...|
|                                               review to come after i get some new year s eve dinner|
|again this was very entertaining to read gives you adding insight into the side characters lives ...|
|                                                                  irial and niall what else is there|
+----------------------------------------------------------------------------------------------------+
only showing top 5 rows


In [0]:
import emoji

# Function to replace emojis
def replace_emoji(text):
    if text is None:
        return ""
    return emoji.replace_emoji(text, replace="<EMOJI>")

emoji_udf = udf(replace_emoji, StringType())
train_df = train_df.withColumn("review_text", emoji_udf(col("review_text")))
train_df.select("review_text").show(5, truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                                         review_text|
+----------------------------------------------------------------------------------------------------+
|this takes place after ink exchange also it focuses on niall s and irial s relationship niall is ...|
|love melissa marrs wicked lovely books and this novella did not let me down very interesting and ...|
|                                               review to come after i get some new year s eve dinner|
|again this was very entertaining to read gives you adding insight into the side characters lives ...|
|                                                                  irial and niall what else is there|
+----------------------------------------------------------------------------------------------------+
only showing top 5 rows


In [0]:
from pyspark.sql.functions import size, split
# Create basic text features
train_df = train_df.withColumn("review_length_chars", length(col("review_text")))
train_df = train_df.withColumn("review_length_words", size(split(col("review_text"), " ")))

# Filter out very short reviews (<10 characters)
train_df = train_df.filter(col("review_length_chars") >= 10)
train_df.select("review_text", "review_length_chars", "review_length_words").show(5, truncate=100)

+----------------------------------------------------------------------------------------------------+-------------------+-------------------+
|                                                                                         review_text|review_length_chars|review_length_words|
+----------------------------------------------------------------------------------------------------+-------------------+-------------------+
|it takes a compelling book to stand up to my current schedule the opposite of dark is such a book...|                880|                172|
|the title of this book gives you something to think about engaging and exciting from the first pa...|                249|                 48|
|everyone who meets charlie pope thinks he looks like a maniac more essentially he is a maniac he ...|               1462|                271|
|adventurous exciting and heart warming i loved the story of the boy who wanted to be a man little...|                482|                 98|

In [0]:
# Install nltk
subprocess.check_call([sys.executable, "-m", "pip", "install", "nltk", "-q"])
import nltk


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [0]:
pip install sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/44.0 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/488.0 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.0/488.0 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/899.7 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/899.7 MB 291.7 MB/s eta 0:00:04
   ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/899.7 MB 228.6 MB/s eta 0:00:04
   ━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/899.7 MB 211.9 MB/s eta 0:00:05
   ━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.5/899.7 MB 179.6 MB/s eta 0:00:05
   ━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/899.7 MB 144.3 MB/s eta 0:00:06
   ━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/899.7 MB 245.9 MB/s eta 0:00:04
   ━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/899.7 MB 205.9 MB/s eta 0:00:05
   ━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/899.7 MB 1

In [0]:
!pip install textblob textstat


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/624.3 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.3/624.3 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/176.4 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.1 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 2.1/2.1 MB 122.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 59.2 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
#Import all required libraries

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import (
    col, year, month, dayofweek, to_timestamp,
    length, size, split, udf
)

from pyspark.sql.types import (
    StringType, DoubleType, FloatType, ArrayType
)
from pyspark.ml.feature import (
    Tokenizer, StopWordsRemover, CountVectorizer, IDF
)
from pyspark.ml import Pipeline

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
import textstat
from sentence_transformers import SentenceTransformer

import numpy as np
import pandas as pd
import re

from sklearn.feature_extraction.text import TfidfVectorizer


In [0]:
nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()

def get_sentiment_pos(text):
    if not text:
        return 0.0
    return analyzer.polarity_scores(text)['pos']

def get_sentiment_neu(text):
    if not text:
        return 0.0
    return analyzer.polarity_scores(text)['neu']

def get_sentiment_neg(text):
    if not text:
        return 0.0
    return analyzer.polarity_scores(text)['neg']

def get_sentiment_compound(text):
    if not text:
        return 0.0
    return analyzer.polarity_scores(text)['compound']

from pyspark.sql import functions as F

sentiment_train = (
    train_df
    .withColumn("sentiment_pos", F.udf(get_sentiment_pos, DoubleType())("review_text"))
    .withColumn("sentiment_neu", F.udf(get_sentiment_neu, DoubleType())("review_text"))
    .withColumn("sentiment_neg", F.udf(get_sentiment_neg, DoubleType())("review_text"))
    .withColumn("sentiment_compound", F.udf(get_sentiment_compound, DoubleType())("review_text"))
)

display(
    sentiment_train.select(
        "review_text",
        "sentiment_pos",
        "sentiment_neu",
        "sentiment_neg",
        "sentiment_compound"
    ).limit(10)
)

sentiment_train.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@goodreadsreviews60104758.dfs.core.windows.net/Gold_layer/feature_v2/sentiment_train/"
)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


review_text,sentiment_pos,sentiment_neu,sentiment_neg,sentiment_compound
it takes a compelling book to stand up to my current schedule the opposite of dark is such a book right now the only chance i get to read is when i m taking my son around to deliver his papers on flyer days he can do the other days on his bike despite the interruptions i not only kept coming back to debra s story i found times to steal away from other stuff so i could read some more the protagonist casey holland would sympathize with my plight she has to investigate her father s deaths yes there were two of them while finding a purse snatcher on the buses and looking out for her almost stepmother and stepsister casey s day job with transit security is one of many perks of the read bounty hunters private investigators and police detectives abound but how many books have transit cops only one that i know of and i ll be picking up the next book in the series to read more,0.05,0.937,0.013,0.4641
the title of this book gives you something to think about engaging and exciting from the first page debra has created a mystery that keeps you reading who knew buses had security and what is the opposite of dark you need to read the book to find out,0.189,0.811,0.0,0.8402
everyone who meets charlie pope thinks he looks like a maniac more essentially he is a maniac he was given a hospital term for a rape and aggravated assault and he is suspected in three more assaults and two murders in fact he is guilty of those cases as well but he isn t volunteering any information about them eight years in a psychiatric hospital was a bore although he met interesting and like minded maniacs who taught him how to abuse and kill with more efficiency he isn t having much fun since he was released though he is a garbageman and he is sick of the stink which no shower will dissipate worse there are so many pretty girls walking around just flaunting themselves so many pretty girls lucas davenport head of minnesota s bureau of criminal apprehension bca is on his own his children and wife weather are in england she had gotten a fellowship in maxilla facial surgery and they had decided to bring the whole family except lucas to london when he was a bachelor he never noticed how quiet a silent house could be now he is feeling uncomfortable with the loneliness but he isn t allowed to linger over his morning coffee long the phone is ringing her body is discovered on a bank stripped naked her skin appears to be striped with bloody lines whipped with wires and her throat has been deeply cut a maniac appears to be on the loose this is in the lucas davenport series it can be read without having read the earlier books in the series,0.089,0.665,0.247,-0.9945
adventurous exciting and heart warming i loved the story of the boy who wanted to be a man little roger is a young man with dreams to do what he loves he has good family values very goal oriented and loves his family i like the bits of french within the story gives a good feel of his ethnic background in a nice touch it has a good lesson to be learned in life and growing up it s an excellent read for adults and children as well i won this book in the goodreads giveaway for free,0.425,0.575,0.0,0.9937
my review http books paradeindah blogspot co,0.0,1.0,0.0,0.0
another dated yet useful book,0.42,0.58,0.0,0.4404
this book has easily made it to the top of my top ten books list as i read it it becomes a great parental guide a resource a resource for other great authors an inspiration the thoughts are refreshing in a confused world with commentary on what it is to be a real man and a real woman the new frontier and how to recognize greatness to name a few i thoroughly enjoyed reading re reading looking up words authors ideas latin writing quotes and thoughts and even recording epiphanies inspired by the author i ve not enjoyed a book this much ever it has been an education in only pages a must read it is out of print and harder to find except on the i

In [0]:
# Load the sentiment-enhanced dataset and extract TF-IDF features using a Spark ML pipeline.

sentiment_train = spark.read.format("delta").load(
    "abfss://lakehouse@goodreadsreviews60104758.dfs.core.windows.net/Gold_layer/feature_v2/sentiment_train/"
)

tokenizer = Tokenizer(inputCol="review_text", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")

cv = CountVectorizer(
    inputCol="filtered_words",
    outputCol="raw_features",
    vocabSize=5000
)

idf = IDF(
    inputCol="raw_features",
    outputCol="tfidf_features"
)

pipeline = Pipeline(stages=[tokenizer, remover, cv, idf])
tfidf_model = pipeline.fit(sentiment_train)
tfidf_train = tfidf_model.transform(sentiment_train)

tfidf_train.cache()

display(tfidf_train.select("review_text", "tfidf_features").limit(10))




Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

🏃 View run silent-dog-531 at: https://adb-2076600740548790.10.azuredatabricks.net/ml/experiments/867946936295008/runs/f01da947b415420d86de62ec054c5170
🧪 View experiment at: https://adb-2076600740548790.10.azuredatabricks.net/ml/experiments/867946936295008


review_text,tfidf_features
murdock ability to build tension and then satisfactorily and realistically release ease that tension is so great there were moments where dj slipped back into old habits that seemed to have been resolved in dairy queen but that s how real life works people learn people slip when faced with new obstacles and old habits die hard the fact that dj not only solidifies her lessons from the last book but makes leaps forward in this one is inspiring the emphasis on work ethic and that the work that goes into making things happen aren t just glossed over is also so refreshing i ve been going through my own struggles the past few months well the last few years really and this series has truly been inspiring even though i m a little over a decade beyond dj s life stage there were still things to be learned or reminded of from this series and the schwenk family the audiobook performance once again was great the accent has calmed down a lot though i think what s actually happened is that i ve gotten used to it i m excited for the next book,"Map(vectorType -> sparse, length -> 5000, indices -> List(0, 4, 6, 13, 14, 15, 19, 21, 22, 23, 26, 28, 33, 38, 39, 43, 44, 45, 46, 50, 54, 71, 73, 92, 94, 102, 103, 108, 114, 123, 126, 159, 168, 219, 239, 241, 248, 266, 296, 300, 314, 318, 328, 373, 380, 629, 666, 695, 734, 824, 851, 889, 939, 946, 1021, 1073, 1186, 1276, 1581, 1588, 1771, 1831, 1921, 1967, 2403, 3390, 3960, 4120, 4126, 4506, 4555, 4760), values -> List(1.2694777983760315, 1.2002947439224234, 1.475454462959383, 3.769753530991031, 3.7477054144857695, 1.7851871343448869, 4.008950038891066, 1.9298076273928535, 1.9587344548928536, 3.808665885462936, 2.0181209632311408, 2.0909710560466648, 4.548324231512576, 2.202074076990847, 4.607813579667527, 2.3294863461072692, 4.625817894920729, 2.3915216505601244, 4.7271184386583505, 2.378267716268987, 2.3860199218927765, 2.4595093698019137, 2.7298330774684576, 5.437878176189364, 2.7489464702083737, 2.734372884935271, 2.7881468565422693, 5.54987648078237, 2.8016977019519, 5.84368516738014, 2.890158960427814, 3.1203533265164873, 3.110111110171183, 4.19859988509696, 3.2418657368129633, 3.347677933917693, 3.381469380630596, 3.397401475934782, 3.4976423210447476, 3.487246370670012, 3.549029755104705, 3.5432250034681942, 3.607206053670909, 3.715865212248872, 3.77868725905693, 4.143851014933908, 4.2219082298530966, 4.277245444443934, 4.31155767432907, 4.431374303788305, 8.933657189180003, 4.498690146074667, 4.8488451444211895, 4.511489365417423, 4.658741226437388, 4.693309375766023, 4.959754955979405, 4.856698796408527, 5.0918984206273405, 5.136612649927071, 5.296295373334133, 5.336318240887844, 10.62639914109322, 5.3547078580113885, 5.5726633040605895, 5.96611986197869, 6.183563113773462, 6.19433069164443, 6.215453901839647, 6.4310632035995186, 6.419432762850294, 13.062449599952053))"
what amazes me most is the title not you might go to the moon not wouldn t it be cool to go to the moon no it conveyed utter certainty you will go to the moon kind of sad now,"Map(vectorType -> sparse, length -> 5000, indices -> List(69, 122, 161, 376, 447, 523, 761, 1720, 4691), values -> List(7.44994634427425, 2.895389952668507, 3.0836709151662665, 3.7172615611223776, 3.8379245709403067, 4.018740017141402, 4.38474284407, 16.544103646070386, 6.356314954620812))"
this book was sent to me by daniel to review and i really enjoyed it it s a fantasy novel that will definitely warm your heart it is centered around two young adults tasha and cade cade has survived a terrible accident that scarred him and killed their parents this has turned tasha into an emotional wreck trying to commit suicide multiple times there is only one place they can be happy and that is dream dream is a beautiful place where our dreams are made it has its own pantheon group of gods and has a lot of different landscapes some of which you are bound to have seen in one of your dreams as a reader in my case that was the crystal caves 

In [0]:
tfidf_train.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@goodreadsreviews60104758.dfs.core.windows.net/Gold_layer/feature_v2/tfidf_train/"
)

In [0]:

embedded_train = spark.read.format("delta").load(
    "abfss://lakehouse@goodreadsreviews60104758.dfs.core.windows.net/Gold_layer/feature_v2/embedded_train/"
)

def compute_readability(text):
    """
    Compute Flesch Reading Ease score for a given review.
    Returns 0.0 if the text is empty or invalid.
    """
    if not text:
        return 0.0
    try:
        return float(textstat.flesch_reading_ease(text))
    except:
        return 0.0

readability_udf = udf(compute_readability, DoubleType())

embedded_train = embedded_train.withColumn(
    "readability_score", readability_udf("review_text")
)

display(embedded_train.select("review_text", "readability_score").limit(5))


review_text,readability_score
murdock ability to build tension and then satisfactorily and realistically release ease that tension is so great there were moments where dj slipped back into old habits that seemed to have been resolved in dairy queen but that s how real life works people learn people slip when faced with new obstacles and old habits die hard the fact that dj not only solidifies her lessons from the last book but makes leaps forward in this one is inspiring the emphasis on work ethic and that the work that goes into making things happen aren t just glossed over is also so refreshing i ve been going through my own struggles the past few months well the last few years really and this series has truly been inspiring even though i m a little over a decade beyond dj s life stage there were still things to be learned or reminded of from this series and the schwenk family the audiobook performance once again was great the accent has calmed down a lot though i think what s actually happened is that i ve gotten used to it i m excited for the next book,-107.79461538461535
what amazes me most is the title not you might go to the moon not wouldn t it be cool to go to the moon no it conveyed utter certainty you will go to the moon kind of sad now,66.83000000000001
this book was sent to me by daniel to review and i really enjoyed it it s a fantasy novel that will definitely warm your heart it is centered around two young adults tasha and cade cade has survived a terrible accident that scarred him and killed their parents this has turned tasha into an emotional wreck trying to commit suicide multiple times there is only one place they can be happy and that is dream dream is a beautiful place where our dreams are made it has its own pantheon group of gods and has a lot of different landscapes some of which you are bound to have seen in one of your dreams as a reader in my case that was the crystal caves but there is a problem called phobetor lord of nightmare he has planned to take over dream and turn it into a nightmare everywhere he goes he is followed by monsters and horrors unimaginable cade is recruited by windwhisper to challenge phobetor and tasha secretly joins him in the form of a mouse i really enjoyed reading this book because it reminded me of the fantasy books i used to read like the neverending story throughout the book i kept on seeing hidden references to this book especially when the silkworms were introduced arenson has created a very rich world with a solid religion he has clearly put thought into how such a group of gods should be structured taking notes i think of the greek gods and how they are interlinked i really liked the way he combined good and evil within these making the bad guys spring from the good the pegacats starlight and harmony were an adorable addition to the gods they really brought light to certain scenes what arenson did really well was describing his character tasha in the beginning she is depressed and can t deal with normal life in the book she is a constant companion to cade who does the biggest part of the travelling because tasha is now a mouse as a reader you can feel her character grow and mature and in the end you really have a bond with her another character that goes through a strong character development is moonlight the princess of the begemmed city in the beginning although nice she is a bit spoiled girl and easily scared in the end after having fought and almost died she is strong and clearly a woman overall arenson has a significant talent for story writing and knows how to guide a reader through a story and keep them interested maybe he had a little bit too many ideas for this book making it seem rushed in some bits yet all of this is only positive it is better to have maybe too many good ideas instead of only two that are boring,-399.4502087682671
stars the gods of dream by daniel arenson is a safe young adult oriented fairytale it is a bit on the dark side but is very li

In [0]:
# Load the TF-IDF dataset and generate semantic embeddings using the SentenceTransformer model.

tfidf_train = spark.read.format("delta").load(
    "abfss://lakehouse@goodreadsreviews60104758.dfs.core.windows.net/Gold_layer/feature_v2/tfidf_train/"
)

model_name = "all-MiniLM-L6-v2"

def embed_text(text):
    if not text:
        return None
    global model
    try:
        model
    except NameError:
        from sentence_transformers import SentenceTransformer
        model = SentenceTransformer(model_name)
    return model.encode(text).tolist()

embed_udf = udf(embed_text, ArrayType(FloatType()))
embedded_train = tfidf_train.withColumn("bert_embedding", embed_udf("review_text"))

display(embedded_train.select("review_text", "bert_embedding").limit(10))

embedded_train.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@goodreadsreviews60104758.dfs.core.windows.net/Gold_layer/feature_v2/embedded_train/"
)


review_text bert_embedding murdock ability to build tension and then satisfactorily and realistically release ease that tension is so great there were moments where dj slipped back into old habits that seemed to have been resolved in dairy queen but that s how real life works people learn people slip when faced with new obstacles and old habits die hard the fact that dj not only solidifies her lessons from the last book but makes leaps forward in this one is inspiring the emphasis on work ethic and that the work that goes into making things happen aren t just glossed over is also so refreshing i ve been going through my own struggles the past few months well the last few years really and this series has truly been inspiring even though i m a little over a decade beyond dj s life stage there were still things to be learned or reminded of from this series and the schwenk family the audiobook performance once again was great the accent has calmed down a lot though i think what s actually happened is that i ve gotten used to it i m excited for the next book List(-0.011413713, -0.15651837, -0.008154633, -0.0429973, 0.038122803, 0.08922089, 0.037097976, 0.033979602, 2.868407E-4, -0.07116059, -0.056054283, 0.04381842, -0.025442198, -0.05410095, -0.05179741, 0.03922987, 0.016219923, 0.03265849, 0.060782127, -0.002740183, -0.04420681, -0.018052567, 0.019951375, 0.07493505, -0.078567475, 0.032376498, -0.016790617, 0.044450834, 0.010189854, -0.036761086, 0.03904736, 0.091190375, -0.027451951, -0.03657599, -0.019784285, 0.114370845, 0.0059212334, -0.0019923637, 0.008310411, -0.027935436, 0.018683452, -0.06323252, 0.037750963, -0.04068364, -0.01954057, -0.048068367, -8.6768676E-4, -0.0799476, 0.056282405, -0.077360034, -0.0778345, -0.080890864, 0.08767831, -0.07754332, 0.03735778, 0.053407174, -0.010315544, 0.1080051, 0.0045711845, -0.09348798, -0.032105077, -0.07052871, -0.027674943, 0.0024167928, 0.027727693, -0.034654606, 3.9921808E-5, 0.070734985, 0.004072102, 0.06189673, -0.11437906, -0.049204886, 0.018190864, -0.030508831, 0.0068791187, 0.076571085, 0.014600486, -0.0835921, 0.034686476, -5.268753E-4, 0.035247885, -0.02443262, -0.03436358, -0.042274732, -0.07927517, -0.02636681, 0.01256574, -0.02522435, 0.02361284, -0.065329075, 0.04768643, 0.026189348, 0.020534884, -0.027708832, 0.06530576, 0.06043427, -0.054002468, 0.044761263, -0.0496164, 0.112445585, -0.030826574, 0.038465858, -4.1627605E-4, -0.08348866, 0.030816395, -0.10055481, 0.048021644, -0.007121133, -0.03438417, -0.021641899, -0.035383042, 0.013991621, -0.03625929, 0.030632077, 0.07842581, 0.08329875, 0.020107046, 0.058567274, -0.039310653, 0.061035756, 0.029832084, 0.04388182, -0.036309816, 0.016515832, -0.040387146, -0.019818578, 0.021185245, 8.094334E-33, -0.03480999, -0.019976178, 0.034931466, 0.07094067, 0.10768898, -0.039753698, -5.9642474E-4, 0.050141606, -0.03132978, 0.043011792, 0.05531979, 0.068750545, -0.045004036, -0.03762231, -0.0696152, 0.0070733824, -0.13305674, -0.065179504, -0.0346152, 0.03455834, -0.02588102, 0.042242367, 0.0046786317, 0.0017490863, -0.008380427, 0.028814701, 0.029924592, -0.0054632146, 0.07187098, 0.037232704, -0.10564526, 0.002707365, -0.11668376, -0.11730444, -0.1576426, 0.026945116, -0.036164623, -0.043951422, 0.04314806, -0.016517779, -0.08143172, -0.019783856, 0.004335155, -0.035046816, -0.048932884, 0.07329831, 0.09706963, -0.0011761187, -0.012687626, 0.011419672, 0.019397836, -0.12745146, -0.048998136, -0.0056668306, -0.051062297, -0.057499062, -0.017133651, -0.004590799, 0.03578714, 0.022613348, 0.078202456, -0.0138652865, 0.0023774256, -0.03712374, 0.022675019, 0.017546536, 0.004480794, 0.025018558, -0.019282058, -0.023281606, -0.06994546, -0.036142904, -0.027996512, -0.057144612, 0.049868863, -0.0072366353, -0.07097012, -0.099952154, -0.027282972, 0.04325483, 0.0059522707, 0.078488305, -0.032244835, 0.022954557, -0.013094371, -0.05514721, -0.0078077386, -0.054096147, 0.0033771342, 0.066922925, -0.04298866, -0.0513127, 0.03491475

In [0]:
# Textblob sentiment analysis score between 0 and 1 (additional feature)

def compute_subjectivity(text):
    if not text:
        return 0.0
    return float(TextBlob(text).sentiment.subjectivity)

subjectivity_udf = udf(compute_subjectivity, DoubleType())

embedded_train = embedded_train.withColumn(
    "subjectivity", subjectivity_udf("review_text")
)

display(embedded_train.select("review_text", "subjectivity").limit(5))


review_text,subjectivity
murdock ability to build tension and then satisfactorily and realistically release ease that tension is so great there were moments where dj slipped back into old habits that seemed to have been resolved in dairy queen but that s how real life works people learn people slip when faced with new obstacles and old habits die hard the fact that dj not only solidifies her lessons from the last book but makes leaps forward in this one is inspiring the emphasis on work ethic and that the work that goes into making things happen aren t just glossed over is also so refreshing i ve been going through my own struggles the past few months well the last few years really and this series has truly been inspiring even though i m a little over a decade beyond dj s life stage there were still things to be learned or reminded of from this series and the schwenk family the audiobook performance once again was great the accent has calmed down a lot though i think what s actually happened is that i ve gotten used to it i m excited for the next book,0.4247373737373738
what amazes me most is the title not you might go to the moon not wouldn t it be cool to go to the moon no it conveyed utter certainty you will go to the moon kind of sad now,0.8099999999999999
this book was sent to me by daniel to review and i really enjoyed it it s a fantasy novel that will definitely warm your heart it is centered around two young adults tasha and cade cade has survived a terrible accident that scarred him and killed their parents this has turned tasha into an emotional wreck trying to commit suicide multiple times there is only one place they can be happy and that is dream dream is a beautiful place where our dreams are made it has its own pantheon group of gods and has a lot of different landscapes some of which you are bound to have seen in one of your dreams as a reader in my case that was the crystal caves but there is a problem called phobetor lord of nightmare he has planned to take over dream and turn it into a nightmare everywhere he goes he is followed by monsters and horrors unimaginable cade is recruited by windwhisper to challenge phobetor and tasha secretly joins him in the form of a mouse i really enjoyed reading this book because it reminded me of the fantasy books i used to read like the neverending story throughout the book i kept on seeing hidden references to this book especially when the silkworms were introduced arenson has created a very rich world with a solid religion he has clearly put thought into how such a group of gods should be structured taking notes i think of the greek gods and how they are interlinked i really liked the way he combined good and evil within these making the bad guys spring from the good the pegacats starlight and harmony were an adorable addition to the gods they really brought light to certain scenes what arenson did really well was describing his character tasha in the beginning she is depressed and can t deal with normal life in the book she is a constant companion to cade who does the biggest part of the travelling because tasha is now a mouse as a reader you can feel her character grow and mature and in the end you really have a bond with her another character that goes through a strong character development is moonlight the princess of the begemmed city in the beginning although nice she is a bit spoiled girl and easily scared in the end after having fought and almost died she is strong and clearly a woman overall arenson has a significant talent for story writing and knows how to guide a reader through a story and keep them interested maybe he had a little bit too many ideas for this book making it seem rushed in some bits yet all of this is only positive it is better to have maybe too many good ideas instead of only two that are boring,0.6051323676323676
stars the gods of dream by daniel arenson is a safe young adult oriented fairytale it is a bit on the dark side but is very light o

In [0]:
# save all the features by merging everything

embedded_train = spark.read.format("delta").load(
    "abfss://lakehouse@goodreadsreviews60104758.dfs.core.windows.net/Gold_layer/feature_v2/embedded_train/"
)

combined_train = embedded_train
combined_train.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save("abfss://lakehouse@goodreadsreviews60104758.dfs.core.windows.net/Gold_layer/feature_v2/combined_train/")


In [0]:
# Verify the output
combined_train = spark.read.format("delta").load(
    "abfss://lakehouse@goodreadsreviews60104758.dfs.core.windows.net/Gold_layer/feature_v2/combined_train/"
)

# Show the first 20 rows
combined_train.show()


+--------+--------------+--------------------+--------------------+---------+--------------------+------+--------------------+-------------+-------+--------------+---------------+--------------------+-----------------+------------------+------------------+------------------+-------------------+-------------------+-------------+-------------+-------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| book_id|          name|           review_id|               title|author_id|             user_id|rating|         review_text|language_code|n_votes|date_added_iso|avg_book_rating|num_reviews_per_book|author_avg_rating|min_words_per_book|max_words_per_book|avg_words_per_book|review_length_chars|review_length_words|sentiment_pos|sentiment_neu|sentiment_neg|sentiment_compound|               words|      filtered_words|        raw_features|      tfidf_features|      bert_embedding|
+--------+--------------+-----------